In [80]:
import pandas as pd
import numpy as np
import sqlite3
import re
from sklearn.cluster import KMeans

conn = sqlite3.connect("profiles.db")
cursor = conn.cursor()
cursor.execute("ATTACH DATABASE 'colleges.db' AS colleges")

In [81]:
df = pd.read_sql_query("SELECT * FROM colleges.university_profiles AS c JOIN profiles AS p ON c.school = p.school",conn)
max_index = len(df.index)

In [82]:
def get_type_map(df_column):
    return {t:i for i, t in enumerate(set(df_column))}

def convert_sat_to_act(sat):
    if np.isnan(sat):
        return np.nan
    sat_scores = np.array([990, 1060, 1140, 1210, 1270, 1330, 1390, 1450, 1510, 1560, 1620, 1680, 1740, 1800, 1860, 1920, 1980, 2020, 2080, 2140, 2220, 2290, 2380, 2410])
    return np.where(sat < sat_scores)[0][0] + 13
    
def combine_test_scores(x):
    if not np.isnan(x['act']):
        return x['act']/36.0
    sat = np.array([x['sat_m'], x['sat_r'], x['sat_w']])
    return convert_sat_to_act(3*sat[~np.isnan(sat)].mean())/36.0 if len(sat[np.isnan(sat)]) != 3 else np.nan

def convert_string(x):
    try:
        return int(x)
    except:
        return np.nan

def combine_avg_test_scores(x):
    if not np.isnan(convert_string(x['avg_act'])):
        return convert_string(x['avg_act'])/36.0
    sat = np.array([convert_string(x['sat_math']), convert_string(x['sat_reading']), convert_string(x['sat_writing'])])
    return convert_sat_to_act(3*sat[~np.isnan(sat)].mean())/36.0 if len(sat[np.isnan(sat)]) != 3 else np.nan

def map_statuses(x):
    status_map = {"Denied": 0, "Deferred": np.nan, "Wait-Listed": np.nan, "Accepted": 1, "Will Attend": 1}
    return status_map[x['status']]

def convert_class_rank(x):
    s = str(x['class_rank'])
    if " of " in s:
        return float(s[:s.index(" of ")])/float(s[s.index(" of ") + 4:])
    if "Top " in s:
        return float(s[4:s.index("%")])/100.0
    if "Bottom " in s:
        return float(s[7:s.index("%")])/100.0
    return np.nan

def convert_instate_tuition(x):
    if "<br>" in str(x['cost_attendance']):
        instate = x['cost_attendance'].split("<br>")[0]
        return instate[instate.index("$")+1:]
    else:
        return x['cost_attendance']

def convert_outstate_tuition(x):
    if "<br>" in str(x['cost_attendance']):
        outstate = x['cost_attendance'].split("<br>")[1]
        return outstate[outstate.index("$")+1:]
    else:
        return x['cost_attendance']

def get_tuition(x):
    return x['in_state_tuition'] if x['state'] == x['hs_state'] else x['out_state_tuition']

def map_hs_types(x):
    hs_type_map = {"Public": 0, "Private": 1, "Parochial": 2, "Home": 3}
    return hs_type_map[x['hs_type']]

def map_ins_types(x):
    ins_type_map = {"Public": 0, "Private": 1, "Private for-profit": 1, "&nbsp;": np.nan}
    return ins_type_map[x['institution_type']]

In [83]:
df = df.loc[:,~df.columns.duplicated()]
df.fillna(value=pd.np.nan, inplace=True)
df['avg_act'] = df['avg_act'].map(lambda x: str(x).strip('-'))
df['test_score'] = df.apply(combine_test_scores, axis=1)
df['avg_test_score'] = df.apply(combine_avg_test_scores, axis=1)
df['class_rank'] = df.apply(convert_class_rank, axis=1)
df['institution_type'] = df.apply(map_ins_types, axis=1)
df['in_state_tuition'] = df.apply(convert_instate_tuition, axis=1)
df['out_state_tuition'] = df.apply(convert_outstate_tuition, axis=1)
#df['tuition'] = df.apply(get_tuition, axis=1)
df['status'] = df.apply(map_statuses, axis=1)

for col in ['cost_attendance', 'city', 'sat_m', 'sat_r', 'sat_w', 'act', 'sat_math', 'sat_reading', 'sat_writing', 'avg_act', 'gpa_w']:
    df = df.drop(col, axis=1)
df = df.dropna()#subset=['test_score','avg_test_score','gpa_uw','class_rank','hs_state','hs_type','institution_type'])

df['hs_type'] = df.apply(map_hs_types, axis=1)
replace_columns = ['hs_state', 'state', 'gender', 'school']
for col in replace_columns:
    df[col].replace(get_type_map(df[col]), inplace=True)
    
dfc = df.copy()
df_plain = df.copy()
for col in ['avg_gpa', 'avg_test_score', 'admission_rate','hs_state','school','hs_type','state','average_freshman_aid','faculty_total','international_percent','institution_type','female_percentage','year','gender','status','eaed','legacy','athlete','in_state_tuition','out_state_tuition']:
    dfc = dfc.drop(col, axis=1)

def getNextTableIndex(index,j):
    isValid = False
    while(not isValid):
        try:
            dfc['gpa_uw'][index]
            return index, j+1
        except:
            index += 1

print("Setting up points")
points = []
i = 0
j=0
while i < 97686:
    i, j = getNextTableIndex(i,j)
    point = []
    for col in dfc.columns.values:
        point.append(dfc[col].get(i))
    points.append(point)
    i += 1
print("Done")
        
cluster = KMeans(n_clusters=5, max_iter=500).fit(points)



Setting up points
Done


In [84]:
print("Assigning clusters")
i = 0
j = 0
while i < 97686:
    label = cluster.labels_[j]
    i, j = getNextTableIndex(i,j)
    dfc.set_value(i, "cluster",label)
    i += 1
    
for col in ['gpa_uw', 'class_rank', 'test_score']:
    df = df.drop(col, axis=1)

df = df.join(dfc, how='outer')
df.dropna()

Assigning clusters


/home/marclane/.local/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys


,school,state,avg_gpa,average_freshman_aid,admission_rate,faculty_total,international_percent,institution_type,female_percentage,year,...,eaed,legacy,athlete,avg_test_score,in_state_tuition,out_state_tuition,gpa_uw,class_rank,test_score,cluster
0,383,40,3.30,11104,88,37,1.8,1.0,50.6,2014,...,0,0,0,0.583333,35214,35214,3.00,0.210753,0.555556,3.0
2,599,10,3.52,21500,73,335,4.0,1.0,68.4,2010,...,0,0,0,0.694444,56738,56738,2.80,0.500000,0.583333,1.0
3,599,10,3.52,21500,73,335,4.0,1.0,68.4,2010,...,0,0,0,0.694444,56738,56738,2.50,0.500000,0.583333,1.0
4,599,10,3.52,21500,73,335,4.0,1.0,68.4,2010,...,0,0,0,0.694444,56738,56738,3.90,0.017544,0.805556,0.0
5,599,10,3.52,21500,73,335,4.0,1.0,68.4,2011,...,0,0,0,0.694444,56738,56738,3.20,0.250000,0.777778,3.0
6,599,10,3.52,21500,73,335,4.0,1.0,68.4,2011,...,0,0,0,0.694444,56738,56738,4.00,0.023891,0.777778,0.0
7,599,10,3.52,21500,73,335,4.0,1.0,68.4,2012,...,0,0,0,0.694444,56738,56738,3.60,0.149626,0.861111,4.0
10,599,10,3.52,21500,73,335,4.0,1.0,68.4,2012,...,0,0,0,0.694444,56738,56738,3.60,0.097059,0.694444,4.0
12,599,10,3.52,21500,73,335,4.0,1.0,68.4,2012,...,0,0,0,0.694444,56738,56738,3.80,0.075231,0.694444,0.0
13,599,10,3.52,21500,73,335,4.0,1.0,68.4,2012,...,0,0,0,0.694444,56738,56738,3.90,0.100000,0.638889,0.0


In [90]:
from sklearn.model_selection import train_test_split, KFold, cross_val_predict
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_fscore_support as score

results = []
for i in range(100):
    print(i)
    for label in set(cluster.labels_):

        try:
            k = 1

            dft = df[df.cluster == label]
            dft.drop('cluster', axis=1)
            df0 = dft[dft.status == 0]
            df2 = dft[dft.status == 1].sample(n = int(k*len(df0)))
            df_final = pd.concat([df0, df2])
            X = df_final.drop('status', axis=1)
            y = df_final['status']

            scaler = StandardScaler()
            scaler.fit(X)
            X = scaler.transform(X)


            X_train1, X_test1, y_train1, y_test1 = train_test_split(X, y)
            mlp = MLPClassifier(hidden_layer_sizes=(8,8,8),max_iter=1000)



            # y_pred = cross_val_predict(mlp, X, y, cv = 4)

            # print(classification_report(y,y_pred))

            # logisticRegr = LogisticRegression()

            # y_pred = cross_val_predict(logisticRegr, X, y, cv = 4)

            # print(classification_report(y,y_pred))

            mlp.fit(X_train1,y_train1)
            predictions = mlp.predict(X_test1)
            results.append(score(y_test1,predictions,average=None))
            #print(confusion_matrix(y_test1,predictions))
            #print(classification_report(y_test1,predictions))
        except:
            pass

    # logisticRegr = LogisticRegression()
    # logisticRegr.fit(X_train1, y_train1)
    # score = logisticRegr.score(X_test1, y_test1)
    # print("Logistic Regression Score: " + str(score))

0


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/marclane/.local/lib/python3.6/site-packages/ipykernel_launcher.py:26: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/marclane/.local/lib/python3.6/site-packages/ipykernel_launcher.py:26: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached 

1


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/marclane/.local/lib/python3.6/site-packages/ipykernel_launcher.py:26: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/marclane/.local/lib/python3.6/site-packages/ipykernel_launcher.py:26: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached 

2


...

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/marclane/.local/lib/python3.6/site-packages/ipykernel_launcher.py:26: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/marclane/.local/lib/python3.6/site-packages/ipykernel_launcher.py:26: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached 

98


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/marclane/.local/lib/python3.6/site-packages/ipykernel_launcher.py:26: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/marclane/.local/lib/python3.6/site-packages/ipykernel_launcher.py:26: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached 

99


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/marclane/.local/lib/python3.6/site-packages/ipykernel_launcher.py:26: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/marclane/.local/lib/python3.6/site-packages/ipykernel_launcher.py:26: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached 

In [86]:
classification_report(y_test1,predictions)

'              precision    recall  f1-score   support\n\n         0.0       0.85      0.88      0.86       367\n         1.0       0.88      0.85      0.86       376\n\n   micro avg       0.86      0.86      0.86       743\n   macro avg       0.86      0.86      0.86       743\nweighted avg       0.86      0.86      0.86       743\n'

In [87]:
df[df.cluster == 6]

,school,state,avg_gpa,average_freshman_aid,admission_rate,faculty_total,international_percent,institution_type,female_percentage,year,...,eaed,legacy,athlete,avg_test_score,in_state_tuition,out_state_tuition,gpa_uw,class_rank,test_score,cluster


In [88]:
results = [x for x in results if x[0][0] != 1]

In [92]:
totals = (sum([x[3][0] for x in results]),sum(x[3][1] for x in results))
sums = [([x[0][0]*x[3][0], x[0][1]*x[3][1]],[x[1][0]*x[3][0], x[1][1]*x[3][1]],[x[2][0]*x[3][0], x[2][1]*x[3][1]]) for x in results]
sums = ([(sum([x[0][0] for x in sums])+sum([x[0][1] for x in sums]))/sum(totals)],
           [(sum([x[1][0] for x in sums])+sum([x[1][1] for x in sums]))/sum(totals)],
           [(sum([x[2][0] for x in sums])+sum([x[2][1] for x in sums]))/sum(totals)])
sums

([0.8293028392381234], [0.8282043461685094], [0.8281203694408942])

In [91]:
results

[(array([0.86211901, 0.92822186]),
  array([0.93103448, 0.85692771]),
  array([0.89525245, 0.89115114]),
  array([638, 664])),
 (array([0.5875    , 0.65753425]),
  array([0.65277778, 0.59259259]),
  array([0.61842105, 0.62337662]),
  array([72, 81])),
 (array([0.74786325, 0.82198953]),
  array([0.83732057, 0.72685185]),
  array([0.79006772, 0.77149877]),
  array([209, 216])),
 (array([0.84070796, 0.81683168]),
  array([0.79387187, 0.859375  ]),
  array([0.81661891, 0.83756345]),
  array([359, 384])),
 (array([0.85606061, 0.87694704]),
  array([0.87732919, 0.8556231 ]),
  array([0.86656442, 0.86615385]),
  array([644, 658])),
 (array([0.66176471, 0.70588235]),
  array([0.64285714, 0.72289157]),
  array([0.65217391, 0.71428571]),
  array([70, 83])),
 (array([0.76415094, 0.77934272]),
  array([0.77511962, 0.76851852]),
  array([0.7695962 , 0.77389277]),
  array([209, 216])),
 (array([0.83636364, 0.81284916]),
  array([0.8277635, 0.8220339]),
  array([0.83204134, 0.81741573]),
  array([389